### Parte 1: Analizando Tendencias de Best-Sellers entre Categorías de Producto

**Objetivo**: Comprender la relación entre las categorías de productos y su estatus de best-seller.

1. **Análisis de Tabla Cruzada (Crosstab)**:
    - Crea una tabla cruzada entre la `category` del producto y el estatus `isBestSeller`.
    
    - ¿Existen categorías donde ser best-seller sea más frecuente? 
    	
    	*Pista: una opción es calcular la proporción de productos best-seller para cada categoría y luego ordenar las categorías en función de esta proporción de forma descendente.*


2. **Pruebas Estadísticas**:
    - Realiza una prueba Chi-cuadrado para determinar si la distribución de best-sellers es independiente de la categoría del producto.
    - Calcula el **V de Cramér** para entender la fuerza de la asociación entre el estatus de best-seller y la categoría.

3. **Visualizaciones**:
	- Visualiza la relación entre las categorías de productos y el estatus de best-seller utilizando un gráfico de barras apiladas.

In [ ]:
import pandas as pd
import csv
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats

In [ ]:
df = pd.read_csv(r"C:/Users/Elías/Desktop/Cajon de Sastre/Bootcamp/GitHub/lab-eda-bivariate/amz_uk_price_prediction_dataset.csv")
df.head()

1. **Análisis de Tabla Cruzada (Crosstab)**:

In [ ]:
# Crea una tabla cruzada entre la `category` del producto y el estatus `isBestSeller`

tabla_cruzada = pd.crosstab(df['category'], df['isBestSeller'])
tabla_cruzada

In [ ]:
# Añadir columna con la proporción
proporcion = (tabla_cruzada[True] / tabla_cruzada.sum(axis=1)).sort_values(ascending=False)
proporcion

In [ ]:
# Tabla completa ordenada por % de best-sellers
tabla_final = tabla_cruzada.copy()
tabla_final['best_seller_ratio'] = proporcion

tabla_final.sort_values('best_seller_ratio', ascending=False)

Sí. Al calcular la proporción de productos best-seller dentro de cada categoría, se observa que algunas categorías tienen un porcentaje significativamente mayor de productos con estatus “best-seller” que otras.
Estas categorías aparecen en las primeras posiciones del ranking (best_seller_ratio), lo que indica que tienen una mayor probabilidad de producir productos best-seller.

2. **Pruebas Estadísticas**:

In [ ]:
from scipy.stats import chi2_contingency
import numpy as np

In [ ]:
# Prueba Chi-cuadrado
chi2, p, dof, expected = chi2_contingency(tabla_cruzada)

print("Chi-cuadrado:", chi2)
print("p-value:", p)
print("Grados de libertad:", dof)

La distribución de productos best-seller no es independiente de la categoría.
Es decir, la categoría influye en la probabilidad de ser best-seller.

In [ ]:
# V de Cramér
n = tabla_cruzada.sum().sum()
phi2 = chi2 / n
r, k = tabla_cruzada.shape

In [ ]:
phi2_corr = max(0, phi2 - ((k - 1)*(r - 1))/(n - 1))
r_corr = r - ((r - 1)**2)/(n - 1)
k_corr = k - ((k - 1)**2)/(n - 1)
v_cramer = np.sqrt(phi2_corr / min((k_corr - 1), (r_corr - 1)))

print("V de Cramér:", v_cramer)

Como category suele tener muchas clases, lo normal es que la relación sea débil pero significativa.

3. **Visualizaciones**:

In [ ]:
proporcion = (tabla_cruzada[True] / tabla_cruzada.sum(axis=1))

plt.figure(figsize=(10, 12))
proporcion.sort_values().plot(
    kind="barh"
)

plt.title("Porcentaje de productos Best-Seller por Categoría")
plt.xlabel("Proporción de Best-Sellers")
plt.ylabel("Categoría")
plt.tight_layout()
plt.show()

### Part 2: Exploring Product Prices and Ratings Across Categories and Brands

**Objective**: Investigate how different product categories influence product prices.

0. **Preliminary Step: Remove outliers in product prices.**

	For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.
	
	*Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.*

1. **Violin Plots**:
    - Use a violin plot to visualize the distribution of `price` across different product `categories`. Filter out the top 20 categories based on count for better visualization.
    - Which product category tends to have the highest median price? Don't filter here by top categories.

2. **Bar Charts**:
    - Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
    - Which product category commands the highest average price? Don't filter here by top categories.

3. **Box Plots**:
    - Visualize the distribution of product `ratings` based on their `category` using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
    - Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

0. **Preliminary Step: Remove outliers in product prices.**

In [ ]:

df_clean = df.copy()

Q1 = df_clean['price'].quantile(0.25)
Q3 = df_clean['price'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_clean = df_clean[(df_clean['price'] >= lower) & (df_clean['price'] <= upper)]

print("Productos tras eliminar outliers:", len(df_clean))

1. **Violin Plots**:

In [ ]:
# Top 20 categorías por número de productos
top20 = df_clean['category'].value_counts().head(20).index
df_top20 = df_clean[df_clean['category'].isin(top20)]

plt.figure(figsize=(14, 8))
sns.violinplot(data=df_top20, x="category", y="price", inner="quartile")
plt.xticks(rotation=90)
plt.title("Distribución del precio por categoría (Top 20)")
plt.tight_layout()
plt.show()

In [ ]:

# Categoría con mayor mediana (sin filtrar)
median_prices = df_clean.groupby("category")["price"].median().sort_values(ascending=False)
print("\nCategoría con la MAYOR mediana de precio:", median_prices.index[0])

2. **Bar Charts**:

In [ ]:
top10 = df_clean['category'].value_counts().head(10).index
df_top10 = df_clean[df_clean['category'].isin(top10)]

mean_prices = df_top10.groupby("category")["price"].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
mean_prices.plot(kind="bar")
plt.title("Precio medio por categoría (Top 10)")
plt.ylabel("Precio medio")
plt.tight_layout()
plt.show()

In [ ]:
# Categoría con mayor precio medio (sin filtrar)
mean_all = df_clean.groupby("category")["price"].mean().sort_values(ascending=False)
print("Categoría con el MAYOR precio medio:", mean_all.index[0])

3. **Box Plots**:

In [ ]:
top10_ratings = df_clean['category'].value_counts().head(10).index
df_top10_ratings = df_clean[df_clean['category'].isin(top10_ratings)]

plt.figure(figsize=(12, 8))
sns.boxplot(data=df_top10_ratings, x="category", y="ratings")
plt.xticks(rotation=90)
plt.title("Distribución de ratings por categoría (Top 10)")
plt.tight_layout()
plt.show()

# Categoría con mayor mediana de ratings (sin filtrar)
median_ratings = df_clean.groupby("category")["ratings"].median().sort_values(ascending=False)
print("Categoría con MAYOR mediana de rating:", median_ratings.index[0])

### Part 3: Investigating the Interplay Between Product Prices and Ratings

**Objective**: Analyze how product ratings (`stars`) correlate with product prices.

1. **Correlation Coefficients**:
    - Calculate the correlation coefficient between `price` and `stars`.
    - Is there a significant correlation between product price and its rating?
	
2. **Visualizations**:
    - Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
    - Use a correlation heatmap to visualize correlations between all numerical variables.
    - Examine if product prices typically follow a normal distribution using a QQ plot. 

1. **Correlation Coefficients**:

In [ ]:

df_corr = df_clean[['price', 'stars']].dropna()

corr_value = df_corr['price'].corr(df_corr['stars'])
print(f"Coeficiente de correlación entre price y stars: {corr_value:.4f}")

# Interpretación automática
if abs(corr_value) < 0.1:
    print("Interpretación: correlación prácticamente nula.")
elif abs(corr_value) < 0.3:
    print("Interpretación: correlación débil.")
elif abs(corr_value) < 0.5:
    print("Interpretación: correlación moderada.")
else:
    print("Interpretación: correlación fuerte.")

# SCATTER PLOT

plt.figure(figsize=(8, 6))
plt.scatter(df_corr['stars'], df_corr['price'], alpha=0.3)
plt.xlabel("stars")
plt.ylabel("price")
plt.title("Relación entre stars y price")
plt.tight_layout()
plt.show()

# HEATMAP DE CORRELACIONES NUMÉRICAS

numeric_df = df_clean.select_dtypes(include='number')
corr_matrix = numeric_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap (Variables numéricas)")
plt.tight_layout()
plt.show()

2. **Visualizations**:

In [ ]:
plt.figure(figsize=(6, 6))
stats.probplot(df_clean['price'].dropna(), dist="norm", plot=plt)
plt.title("QQ Plot de price (normalidad)")
plt.tight_layout()
plt.show()